In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from pprint import pprint as pp

In [2]:
#Aug 25 - Non-clipping
#Sept 14 - Clipping 
#ALl of August

df1 = pd.read_csv("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Eglin_Analysis\\Eglin - 9-14 CBX1.csv")
df2 = pd.read_csv("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Eglin_Analysis\\Eglin - 9-14 CBX2.csv")
df3 = pd.read_csv("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Eglin_Analysis\\Eglin - 9-14_MTR_POA.csv")

df_index = df1['Date (-05:00)']


In [3]:
all_df = pd.concat([df1.reset_index(drop=True),
                    df2.reset_index(drop=True), 
                    df3.reset_index(drop=True)], axis=1,ignore_index=True)

df_columns = [list(df1.columns),
              list(df2.columns),
              list(df3.columns),]

flatten = lambda nested_list: [item for sublist in nested_list for item in sublist]
all_df.columns = flatten(df_columns)

for i in all_df.columns:
    if "Date (-05:00)" in i:
        new_df = all_df.drop([i], axis=1)


new_df = new_df.set_index([df_index])
new_df.index = pd.to_datetime(new_df.index)
new_df = new_df[~new_df.index.duplicated(keep='first')]
new_df

,Recombiner 101 - Current_String_1 (A),Recombiner 101 - Current_String_2 (A),Recombiner 101 - Current_String_3 (A),Recombiner 101 - Current_String_4 (A),Recombiner 101 - Current_String_5 (A),Recombiner 101 - Current_String_6 (A),Recombiner 101 - Current_String_7 (A),Recombiner 101 - Current_String_8 (A),Recombiner 101 - Current_String_9 (A),Recombiner 101 - Current_String_10 (A),...,Recombiner 207 - Current_String_12 (A),Recombiner 207 - Current_String_13 (A),Recombiner 207 - Current_String_14 (A),Recombiner 207 - Current_String_15 (A),Recombiner 207 - Current_String_16 (A),Recombiner 207 - Current_String_17 (A),Recombiner 207 - Current_String_18 (A),Recombiner 207 - Current_String_19 (A),Meter AC Power (W),Sensors - Irradiance reference (W/m²)
Date (-05:00),,,,,,,,,,,,,,,,,,,,,
2022-09-14 00:05:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.3,...,-0.1,0.0,-0.2,0.2,0.2,0.2,0.2,0.0,0,NaN
2022-09-14 00:10:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.3,-0.2,...,-0.1,0.0,-0.2,0.1,0.2,0.2,0.2,0.0,0,NaN
2022-09-14 00:15:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.3,...,-0.1,0.0,-0.2,0.2,0.2,0.2,0.2,0.0,0,NaN
2022-09-14 00:20:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.3,-0.2,...,-0.1,0.0,-0.2,0.1,0.3,0.2,0.2,0.0,0,NaN
2022-09-14 00:25:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.2,...,-0.1,0.0,-0.2,0.1,0.2,0.2,0.2,0.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-14 23:40:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.2,...,-0.1,0.0,-0.2,0.1,0.2,0.2,0.2,0.0,0,NaN
2022-09-14 23:45:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.3,-0.2,...,-0.1,0.0,-0.2,0.2,0.2,0.2,0.2,0.0,0,NaN
2022-09-14 23:50:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.3,-0.2,...,-0.1,0.0,-0.2,0.1,0.2,0.2,0.2,0.0,0,NaN


In [4]:
''' 
Filters:
        1. No Negative values for CBX
        2. Meter filtered for 10% of MAX for MIN and 90% of MAX for Clipping
        3. Irradiance above 200
        4. Start at 11 am, Peak hours/No Shading
Added Parameters: 
        5. DC Expected = daily_poa*string_count*IMP
        6. Heat Map: Summation(DC_Current)/DC_Expected
'''


for i in new_df.columns:
    if "Recombiner" in i:
        filtered_df1 = new_df[new_df.loc[:,i] > 1]
        
meter_min = 30000000 * 0.10
meter_max = 30000000 * 0.98

# filtered_df3 = filtered_df1[(filtered_df1.loc[:, "Meter AC Power (W)"]>meter_min)&(filtered_df1.loc[:, "Meter AC Power (W)"]<meter_max)&(filtered_df1.loc[:,"Irradiance Reference (Link) (W/m²)"]>200)]
filtered_df3 = filtered_df1[(filtered_df1.loc[:, "Meter AC Power (W)"]>meter_min)&(filtered_df1.loc[:, "Meter AC Power (W)"]<meter_max)&(filtered_df1.loc[:,"Sensors - Irradiance reference (W/m²)"]>200)]

filtered_df3 = new_df.between_time("9:00", "17:00")
filtered_df3[filtered_df3 <= 0.00] = 0


C:\Users\dpinales\AppData\Local\Temp\ipykernel_57064\3029860732.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df3[filtered_df3 <= 0.00] = 0
C:\Users\dpinales\AppData\Local\Temp\ipykernel_57064\3029860732.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df3[filtered_df3 <= 0.00] = 0


In [5]:
# df = pd.read_excel("Filtered Data.xlsx")

cbx = []
cbx_sum = []


for x in filtered_df3:
    if "Recombiner" in x:
        cbx.append(x)
        cbx_sum.append(filtered_df3[x].sum())
        
cbx_data = pd.DataFrame(cbx_sum, index=[cbx], columns=['Sum_of_CBX'])
cbx_data

,Sum_of_CBX
Recombiner 101 - Current_String_1 (A),11257.2
Recombiner 101 - Current_String_2 (A),11273.9
Recombiner 101 - Current_String_3 (A),11223.1
Recombiner 101 - Current_String_4 (A),11505.1
Recombiner 101 - Current_String_5 (A),11563.4
...,...
Recombiner 207 - Current_String_15 (A),11041.2
Recombiner 207 - Current_String_16 (A),9341.8
Recombiner 207 - Current_String_17 (A),8657.9
Recombiner 207 - Current_String_18 (A),8831.9


In [6]:
dc_issues = pd.read_excel("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Eglin_Analysis\\Gulf DC Issues.xlsx")

dc_rating = []
relative_dc = []

for i in dc_issues['DC Rating Power'].values:
    dc_rating.append(i)

max_dc = max(dc_rating)

for i in dc_rating:
    dc = (i/max_dc)
    relative_dc.append(dc)
        

cbx_data["DC_Rating"] = dc_rating
cbx_data["String_count"] = [s for s in dc_issues['string_count']]
cbx_data['Relative_DC'] = relative_dc


_sum = cbx_data['Sum_of_CBX'].values
_dc = cbx_data['Relative_DC'].values
sum_over_dc = _sum/_dc

cbx_data['Sum_of_CBX/Relative_DC'] = sum_over_dc



In [14]:

top_performers = np.argsort(sum_over_dc)[-5:]

top_performers_values = [sum_over_dc[i] for i in top_performers]

_top_med = np.median(top_performers_values)

sum_performer = (sum_over_dc/_top_med)
clipping = np.clip(sum_performer, 0.00,1.00)

cbx_data["Sum/Top_performer"] = clipping

string_count = cbx_data['String_count'].values
down_strings = string_count*(1-clipping)

cbx_data['Estimated_Downstrings'] = down_strings 

daily_loss = filtered_df3.resample("1D").sum()/12000 
daily_loss.loc[:, daily_loss.columns.str.contains("Recombiner")].align(cbx_data['Estimated_Downstrings'], axis=0)

ValueError: cannot join with no overlapping index names

In [30]:
fig_dates = filtered_df3.index
fig_index = cbx_data.index
fig_heatdf = cbx_data.filter(["Estimated_Downstrings", "String_count"])
# fig_heatdf
fig = px.density_heatmap(fig_heatdf, x=fig_heatdf.index.values,y=fig_dates,)
fig.show()


ValueError: All arguments should have the same length. The length of argument `wide_variable_0` is 1, whereas the length of  previously-processed arguments ['y'] is 97

In [47]:
e_downstrings = sum(down_strings)
total_string = sum(string_count)
ratio_strings = e_downstrings/total_string

print(f"The estimated plants DC outage: {ratio_strings}")

The estimated plants DC outage: 0.10485449269970572


In [49]:
inverters = pd.read_csv("data\\Eglin - Nonclipping_8-25 INV.csv")
inverters = inverters.set_index([df_index])
inverters.index = pd.to_datetime(inverters.index)
inverters = inverters[~inverters.index.duplicated(keep='first')]
inverters = inverters.drop("Date (-05:00)", axis=1)
inverters = inverters.between_time("9:00", "17:00")

inv_sum = []
inv = []


for x in inverters.columns:
    if "INV" in x:
        inv.append(x)
        inv_sum.append(inverters[x].sum())
inv_df = pd.DataFrame(inv_sum, index=[inv], columns=['Sum_of_inv'])
inv_df

,Sum_of_inv
INV 101 - (A),20390
INV 102 - (A),20679
INV 103 - (A),19987
INV 104 - (A),20929
INV 105 - (A),20970
INV 106 - (A),21279
INV 107 - (A),20910
INV 201 - (A),20808
INV 202 - (A),21055
INV 203 - (A),19344


In [50]:
'''Summation of CMB/INV'''


sum_of_cbx = cbx_data['Sum_of_CBX']
sum_of_inv = inv_df['Sum_of_inv']

def cmb_summation(a,b):
    total = (a/b)
    return total


cmb1 = []
blk1_sum = []
_summation1 = []

inv1 = []
inv1_blk = []

for inv_range in range(100, 108, 1):
    inv1.append(sum_of_inv.filter(like=str(inv_range), axis=0))

for i in range(1, len(inv1)):
    _inv1 = (sum(inv1[i]))
    inv1_blk.append(_inv1)    

for cmb_range in range(100, 108, 1):
    cmb1.append(sum_of_cbx.filter(like=str(cmb_range), axis=0))

for i in range(1,len(cmb1)):
    _cb1 = (sum(cmb1[i]))
    blk1_sum.append(_cb1)

for (v, n) in zip(blk1_sum, inv1_blk):
    _summation1.append(cmb_summation(v, n))
    

In [52]:
inv2_dc = {
    "INV1",
    "INV2",
    "INV3",
    "INV4",
    "INV5",
    "INV6",
    "INV7",
}


cmb2 = []
blk2_sum = []

inv2 = []
inv2_blk = []

_summation2 = []

for inv_range in range(200, 208, 1):
    inv2.append(sum_of_inv.filter(like=str(inv_range), axis=0))

for i in range(1, len(inv2)):
    _inv2 = (sum(inv2[i]))
    inv2_blk.append(_inv2)    

for cmb_range in range(200, 208, 1):
    cmb2.append(sum_of_cbx.filter(like=str(cmb_range), axis=0))

for i in range(1,len(cmb2)):
    _cb2 = (sum(cmb2[i]))
    blk2_sum.append(_cb2)

for (v, n) in zip(blk2_sum, inv1_blk):
    _summation2.append(cmb_summation(v, n))


In [54]:
total_summation = list(zip(_summation1, _summation2))

cbx_over_inv = pd.DataFrame(total_summation, index=[n for n in inv2_dc.keys()],columns=['BLK1 CBX100-107/INV DC', "BLK2 CBX200-207/INV DC"])
cbx_over_inv

,BLK1 CBX100-107/INV DC,BLK2 CBX200-207/INV DC
INV1,0.978588,0.994262
INV2,0.980125,1.004130
INV3,0.990554,0.957267
INV4,0.986693,1.018204
INV5,0.993224,1.044568
INV6,0.975779,0.983298
INV7,0.984998,1.038465


In [17]:
cbx_data.to_csv("Gulf I - 9-14.csv")

PermissionError: [Errno 13] Permission denied: 'Gulf I - 9-14.csv'